# Sources

1. Skienna

2. https://www.geeksforgeeks.org/longest-common-subsequence-dp-4/

### Fibonacci

In [21]:
import numpy as np
import time 

def fibb_recurse(n):
    """
    
    Time = O(C^N) -- exponential!
    
    """
    
    if n == 0:
        return 0
    if n == 1:
        return 1
    #print(n) 
    return fibb_recurse(n-1) + fibb_recurse(n-2)

n = 38
t1 = time.time()
fibb_recurse(n)
t = (time.time() - t1) / 60.0
print('n = {} took {:.2f} mins'.format(n,t))

n = 38 took 0.18 mins


In [22]:
def fibb_dp(n):
    """
    1. Time = Space = O(N)
    2. In fact, can reduce the space, since we only need previous two values
    """
    F = np.zeros(n+1)
    F[0], F[1] = 0,1
    for i in range(2,n+1):
        F[i] = int(F[i-1] + F[i-2])
    return int(F[-1])

n = 38
t1 = time.time()
fibb_dp(n)
t = (time.time() - t1) / 60.0
print('n = {} took {:.2f} mins'.format(n,t))

n = 38 took 0.00 mins


### Binomial coefficients

In [45]:
from math import factorial

def bin_long(n,k):
    num = factorial(n)
    denom = factorial(k)*factorial(n-k)
    return num / denom


n,k = 120,17
t1 = time.time()
x = bin_long(n,k)
t = (time.time() - t1) / 60.0
print('n = {} took {:.2f} mins'.format(x,t))

n = 1.8991659143539684e+20 took 0.00 mins


In [46]:
def bin_dp(n,k):
    
    #Instantiate
    B = np.zeros((n+1,n+1))
    B[0][0] = 1
    
    #Do DP
    for n1 in range(1,n+1):
        for k1 in range(n1+1):
            B[n1][k1] = B[n1-1][k1-1] + B[n1-1][k1]   #from pascals triangle
    
    return B[n][k]


n,k = 120,17
t1 = time.time()
x = bin_dp(n,k)
t = (time.time() - t1) / 60.0
print('n = {} took {:.2f} mins'.format(x,t))

n = 1.899165914353968e+20 took 0.00 mins


### Approximate string matching

In [4]:
### First do substring matching -- this is a reminder
def match(s,p):
    if len(p) == 0:
        return True
    else:
        for i in range(len(s)):
            if s[i] == p[0]:
                return match(s[i+1:],p[1:])
        return False
    
s = 'this is a test'
p = 'is'
match(s,p)

True

In [169]:
#Recursion
def minDistance(s,p):
    """
    This one is only locally optimal
    """
    
    if len(p) == 0:
        return len(s)
    if len(s) == 0:
        return len(p)
    else:
        if s[-1] == p[-1]:
            return minDistance(s[:-1],p[:-1])
        
        #Need to delete
        if len(s) > len(p):
            return 1 + minDistance(s[:-1],p)
        
        #Need to insert
        if len(s) < len(p):
            return 1 + minDistance(s,p[:-1])
        
        #To to replace
        if len(s) == len(p):
            return 1 + minDistance(s[:-1], p[:-1])
        
        
def minDistance1(s,p):
    """
    This is fully optimal
    """
    if len(p) == 0:
        return len(s)
    if len(s) == 0:
        return len(p)
    else:
        if s[-1] == p[-1]:
            return minDistance(s[:-1],p[:-1])
        else:  # delete, insert, replace
            return 1 + min( minDistance(s[:-1],p), minDistance(s,p[:-1]), minDistance(s[:-1],p[:-1]) )
        
s = 'this is a bad day'
p = 'peck like two birds'
minDistance1(s,p)

16

Above has exponential complexity. Problem is ripe for dynamic programming

In [194]:
#Dynamic programming
def minDistanceDP(s,p):
    """ D(0,0) means both strings are empty
        I assume the 
    """
    
    m,n = len(s), len(p)
    D = np.zeros((m+1,n+1))
    
    #Rest
    for i in range(m+1):
        for j in range(n+1):
            
            #If s is empty, then length of p is answer
            if i == 0:
                D[i][j] = j

            #If p is empty, then length of s is answer
            elif j == 0:
                D[i][j] = i
                
            elif s[i-1] == p[j-1]:
                D[i,j] = D[i-1,j-1]
            
            else:
                delete =  D[i-1,j]
                insert = D[i,j-1]
                sub = D[i-1,j-1]
                D[i,j] =  1 + min(delete,insert,sub)
    return D[m,n]

s = 'is'
p = 'this'
n,m = len(s), len(p)
minDistanceDP(s,p)

2.0

Sweet. Idea: think of the state space / recursive tree

In [216]:
def minDistanceDPwithPath(s,p):
    """ D(0,0) means both strings are empty
        I assume the 
        
        The idea here is that once I have the 
        cost matrix, I can start at the bottom 
        right hand corner and then greedily
        move to the top left
    """
    
    m,n = len(s), len(p)
    D = np.zeros((m+1,n+1))
    
    #Rest
    for i in range(m+1):
        for j in range(n+1):
            
            #If s is empty, then length of p is answer
            if i == 0:
                D[i][j] = j

            #If p is empty, then length of s is answer
            elif j == 0:
                D[i][j] = i
                
            elif s[i-1] == p[j-1]:
                D[i,j] = D[i-1,j-1]
            
            else:
                delete =  D[i-1,j]
                insert = D[i,j-1]
                sub = D[i-1,j-1]
                D[i,j] =  1 + min(delete,insert,sub)
    
    minCost = D[m,n]
    
    #Now find path, greedy from end of path
    position = (m,n)
    path = ''
    names = ['substitution','insertion','deletion']
    while position != (0,0):
        (i,j) = position
        currCost = D[m,n]
        if m == 0:
            neighbours = [(i,j-1)]
        elif n == 0:
            neighbours = [(i-1,j)]
        else:
            neighbours = [(i-1,j-1),(i,j-1),(i-1,j)]
        costs = [D[x] for x in neighbours]
        minCostindex = int(np.argmin(costs))  #non-unique
        position = neighbours[minCostindex]
        if currCost != costs[minCostindex]:
            path += ' ' + names[minCostindex]
        #print(position,neighbours)
    return minCost,path

s = 'this is a test'
p = 'is'
n,m = len(s), len(p)
minCost, path = minDistanceDPwithPath(s,p)
minCost, path

(12.0,
 ' deletion deletion deletion deletion deletion deletion deletion deletion deletion deletion substitution substitution deletion deletion')

### Longest increasing sequence

In [19]:
def longest(s):
    if len(s) == 1:
        return 1
    else:
        x,rest = s[-1],s[:-1]
        if x > max(rest):
            return 1 + longest(rest)
        else:
            return longest(rest)
        
s = [2,4,3,5,1,7,6,9,8]
longest(s)

5

In [281]:
def longestDP(s):
    """
    
    Let l_i = longest sequence in s_0, s_1, ... s_i
    
    Then l_i = 1 + max_{j < i} l_j,  and s_j < s_i
    
    """
    
    l = [0]*(len(s)+1)
    
    for i in range(1,len(s)+1):
        maxl = 0
        for j in range(i):
            if s[j-1] < s[i-1] and l[j] > maxl:   #string has 1 indexing
                maxl = l[j]
        l[i] = 1 + maxl
    return l[-1]

s = [2,4,3,5,1,7,6,9,8]
longestDP(s)

5

### Linear partition (skinenna p295)

### Longest common substring

Problems from https://www.geeksforgeeks.org/top-20-dynamic-programming-interview-questions/ start here

In [51]:
def lcs(x,y):
    
    #Boundary conditions
    if len(x) == 0:
        return 0
    elif len(y) == 0:
        return 0
    
    #Main
    else:
        if x[-1] == y[-1]:
            return 1 + lcs(x[:-1],y[:-1])
        else:
            return max(lcs(x[:-1],y), lcs(x,y[:-1]))
        
    
x, y = 'age', 'agde'
lcs(x,y)

3

In [52]:
def lcsDP(x,y):
    m,n = len(x), len(y)
    D = np.zeros((m+1,n+1))
    for i in range(m+1):
        for j in range(n+1):
            if i == 0 or j == 0:
                D[i,j] = 0
            elif x[i-1] == y[j-1]:
                D[i,j] = 1 + D[i-1,j-1]
            else:
                D[i,j] = max(D[i-1,j],D[i,j-1])
    return int(D[m][n])
                    
x, y = 'age', 'agde'
lcsDP(x,y)

3

### Form subsets -- not finished

In [304]:
def subset(x):
    if len(x) == 1:
        yield [x]
    else:
        for i in range(len(x)):
            first, rest = x[i], x[:i] + x[i+1:]
            print('first,rest = {},{}'.format(first,rest))
            for s in subset(rest):
                yield [first] + s
                
x = 'abc'
list(subset(x))

first,rest = a,bc
first,rest = b,c
first,rest = c,b
first,rest = b,ac
first,rest = a,c
first,rest = c,a
first,rest = c,ab
first,rest = a,b
first,rest = b,a


[['a', 'b', 'c'],
 ['a', 'c', 'b'],
 ['b', 'a', 'c'],
 ['b', 'c', 'a'],
 ['c', 'a', 'b'],
 ['c', 'b', 'a']]

### Count number of ways to cover distance

Have to travel $n$, can jump 1,2,3 steps

In [309]:
def steps(n):
    """
    M(n) = M(n-1) + M(n-2) + M(n-3)
    """
    
    M = [0]*(n+1)
    M[1] = 1
    M[2] = M[1] + 1
    M[3] = M[1] + M[2] + 1 
    for i in range(4,n+1):
        M[i] = M[i-3] + M[i-2] + M[i-1]
    return M[-1]

n = 5
steps(n)

13

### Longest path in a matrix

In [311]:
def path(M):
    """
    M(n) = M(n-1) + M(n-2) + M(n-3)
    """
    
    m,n = len(M), len(M[0])
    D = [[0]*n]*m
    
    #Dynamic programming
    for i in range(m):
        for j in range(n):

    return D


def neighbours(i,j):
    neighbours = [(i+1,j),(i-1,j),(i,j+1),(i,j-1)]
    for n in neighbours:
        if n[0] 

M = [[1,2,9],[5,3,8],[4,6,7]]
path(M)

[[0, 0, 0], [0, 0, 0], [0, 0, 0]]

### Cost of path

In [73]:
#Longest path in matrix

def path(M):
    """
    D(i,j) cost to get to end if started on (i,j)
    """
    
    m,n = len(M), len(M[0])
    D = [[0]*m]*n
    
    D[m-1][n-1] = M[m-1][n-1]
    for i in range(m-1,-1,-1):
        for j in range(n-1,-1,-1):
            if i == m-1 and j == n-1:
                pass
            elif i == m-1:
                D[i][j] = M[i][j] + D[i][j+1]   #right only
            elif j == n-1:
                D[i][j] = M[i][j] + D[i+1][j]  #below only
            else:
                D[i][j] = M[i][j] + min(D[i][j+1],D[i+1][j])  #min(left,above)
    
    return D[0][0]

M = [[1,2,9],[5,3,8],[4,6,7]]
path(M)

19

### 